In [13]:
import numpy as np
import os
import glob
import scipy.ndimage as ndimage
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt

In [17]:
# Load all the images up
directory_path = './data/cells'
image_files = [f for f in glob.glob(os.path.join(directory_path, '*')) if 'Edges' not in os.path.basename(f)]
images = [Image.open(filename) for filename in image_files]

In [18]:
for img in images:
    img.show()